In [3]:
import os
import math
import numpy as np
import pandas as pd
import re
import nglview as nv
import mdtraj as md
import MDAnalysis as mda

A Jupyter Widget

In [6]:
#I/O
wd=os.getcwd()+"/"
npIn='Ag111-26.35_PEG-19.9_zp-0.031.np' #kep this naming format, so NP parameters can be read in in next step
pdbOut=npIn.rsplit('.', 1)[0]+".pdb"
pngOut=npIn.rsplit('.', 1)[0]+".png"

In [7]:
rCore=float(npIn.rsplit('.', 1)[0].rsplit('_')[0].rsplit('-')[1])
rDecor=3.0
core=re.sub("[^a-zA-Z]+", "", npIn.rsplit('.', 1)[0].rsplit('_')[0].rsplit('-')[0])
decor=npIn.rsplit('.', 1)[0].rsplit('_')[1].rsplit('-')[0]

In [8]:
elemDic=    { "Ag"   : "Ag",
               "PEG" : "C"}
typeDic=    {  "Ag"  : "AGM",
               "PEG" : "CPG"}
beadDicRGB= {  "Ag"  : [0.23, 0.27, 0.29],
               "PEG" : [0.98, 0.94, 0.75]}
beadDicHex= {   "Ag" : "#3B444B",
                "PEG": "#FAF0BE"}
beadRDic=   {    "Ag": rCore,
                "PEG": rDecor}

In [9]:
rawNP         = np.genfromtxt(wd+npIn,  delimiter=',' , skip_header=1, dtype=None)
NPcoordList=[]
NPresNameArray=[]
NPRad=[]
if rawNP.size > 1:
    i=1
    for line in rawNP:
        npX, npY, npZ, npR, _, _, _, _, ss, _, _, _ = line
        solid=(((str(ss)).rsplit( "/" )[1]).rsplit( "." ))[0]
        NPresNameArray.append([solid, solid, int(i)])
        NPcoordList.append([float(round(npX,3)*10), float(round(npY,3)*10), float(round(npZ,3)*10)])
        NPRad.append([int(i), solid, float(npR)])
        i=i+1
if rawNP.size ==1:
    NPcoordList.append([0.0, 0.0, 0.0])
    NPresNameArray.append([core,core, 1])
    NPRad.append([1,core, radii])
NPcoord=np.array(NPcoordList)
NPres=np.array(NPresNameArray)
NPRs=np.array(NPRad)    

/tmp/ipykernel_459272/2767191048.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  rawNP         = np.genfromtxt(wd+npIn,  delimiter=',' , skip_header=1, dtype=None)


In [10]:
nameLst=[]
resLst=[]
typeLst=[]
elemLst=[]
coorArr=[]
for i, line in enumerate(NPres):
    typeLst.append(typeDic[line[0]])
    elemLst.append(elemDic[line[0]])
    nameLst.append(line[0])
    resLst.append(line[1].upper())
    coorArr.append(NPcoord[i])
n_residues = len(NPres)
n_atoms = len(NPres)
resindices = np.repeat(range(n_residues), 1)
assert len(resindices) == n_atoms
segindices = [0] * n_residues
u_nano = mda.Universe.empty(n_atoms,
                         n_residues=n_residues,
                         atom_resindex=resindices,
                         residue_segindex=segindices,
                         trajectory=True)
u_nano.add_TopologyAttr('name', typeLst)
u_nano.add_TopologyAttr('type', typeLst)
u_nano.add_TopologyAttr('elements', elemLst)
u_nano.add_TopologyAttr('resname', resLst)
u_nano.add_TopologyAttr('resid', list(range(1, n_residues+1)))
u_nano.add_TopologyAttr('segid', ["NP"])
u_nano.atoms.positions = coorArr
u_nano.atoms.write(wd+pdbOut)

/home/ysubbotina/anaconda3/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:753: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/ysubbotina/anaconda3/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/ysubbotina/anaconda3/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/ysubbotina/anaconda3/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/home/ysubbotina/anaconda3/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarni

In [11]:
view=nv.show_mdanalysis(u_nano)
view.clear_representations()
view._remote_call('setSize', target='Widget', args=['400px','400px'])
view.add_representation(repr_type='spacefill', selection=decor,  opacity=0.25, color=beadDicHex[decor], radius=rDecor*10)
shape = view.shape
shape.add_sphere([ 0.0, 0.0, 0.0 ], beadDicRGB[core], rCore*10, core.upper())
view.update_representation()
view.center()
view.parameters = dict(theme='light')
view.download_image(filename=pngOut , factor=10, transparent=True, trim=True)
view

A Jupyter Widget